# Code for the paper: 

## Efficient and Mobile Deep Learning Architectures for Fast Identification of BacterialStrains in Resource-Constrained Devices

### Architecture: ShuffleNet v2x1.5
### Data: Augmented + Cross-Validation

#### Loading libraries

In [1]:
# Imports here
import os
import torch
import numpy as np
import torch.nn.functional as F
import pandas as pd
import time
import numpy as np

from PIL import Image
from sklearn.metrics import f1_score, precision_score, recall_score, classification_report
from sklearn.model_selection import KFold
from torch import nn
from torch import optim
from torch.utils.data import SubsetRandomSampler
from torchvision import datasets, transforms, models
from pytorch_model_summary import summary

# Archs not in Pytorch
from efficientnet_pytorch import EfficientNet

# External functions
from scripts.utils import *

In [2]:
print(torch.__version__)

import sys
print(sys.version)

1.5.1
3.8.5 (default, Jul 28 2020, 12:59:40) 
[GCC 9.3.0]


## Data paths and hyperparameters

#### Hyperparameters and dataset details. 

In [3]:
# Dataset details
dataset_version = 'original' # original or augmented
img_shape = (224,224)
img_size = str(img_shape[0])+"x"+str(img_shape[1])

# Root directory of dataset
data_dir = '/home/yibbtstll/venvs/pytorch_gpu/CySDeepBacterial/Dataset/DIBaS_augmented/'

train_batch_size = 64
val_test_batch_size = 32
feature_extract = False
pretrained = False
h_epochs = 15
kfolds = 10

## Data preparation and loading

#### Defining transforms and creating dataloaders

In [4]:
# Define transforms for input data
training_transforms = transforms.Compose([transforms.ToTensor(),
                                          transforms.Normalize([0.485, 0.456, 0.406], 
                                                               [0.229, 0.224, 0.225])])

# TODO: Load the datasets with ImageFolder
total_set = datasets.ImageFolder(data_dir, transform=training_transforms)

# Defining folds
splits = KFold(n_splits = kfolds, shuffle = True, random_state = 42)

#### Visualizing the target classes in dataset

In [5]:
train_labels = {value : key for (key, value) in total_set.class_to_idx.items()}
    
print(len(train_labels)) 
print(train_labels)

32
{0: 'Acinetobacter.baumanii', 1: 'Actinomyces.israeli', 2: 'Bacteroides.fragilis', 3: 'Bifidobacterium.spp', 4: 'Clostridium.perfringens', 5: 'Enterococcus.faecalis', 6: 'Enterococcus.faecium', 7: 'Escherichia.coli', 8: 'Fusobacterium', 9: 'Lactobacillus.casei', 10: 'Lactobacillus.crispatus', 11: 'Lactobacillus.delbrueckii', 12: 'Lactobacillus.gasseri', 13: 'Lactobacillus.jehnsenii', 14: 'Lactobacillus.johnsonii', 15: 'Lactobacillus.paracasei', 16: 'Lactobacillus.plantarum', 17: 'Lactobacillus.reuteri', 18: 'Lactobacillus.rhamnosus', 19: 'Lactobacillus.salivarius', 20: 'Listeria.monocytogenes', 21: 'Micrococcus.spp', 22: 'Neisseria.gonorrhoeae', 23: 'Porfyromonas.gingivalis', 24: 'Propionibacterium.acnes', 25: 'Proteus', 26: 'Pseudomonas.aeruginosa', 27: 'Staphylococcus.aureus', 28: 'Staphylococcus.epidermidis', 29: 'Staphylococcus.saprophiticus', 30: 'Streptococcus.agalactiae', 31: 'Veionella'}


## Model definition and inicialization

#### Freezing pre-trained parameters, finetunning the classifier to output 32 classes.

In [6]:
# Freeze pretrained model parameters to avoid backpropogating through them
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        print("Setting grad to false.")
        for param in model.parameters():
            param.requires_grad = False
    
    return model

def get_device():
    # Model and criterion to GPU
    if torch.cuda.is_available():
        return 'cuda'
    else:
        return 'cpu'

In [7]:
models.shufflenet_v2_x1_0(pretrained=False)

ShuffleNetV2(
  (conv1): Sequential(
    (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (stage2): Sequential(
    (0): InvertedResidual(
      (branch1): Sequential(
        (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=24, bias=False)
        (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): Conv2d(24, 58, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (3): BatchNorm2d(58, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (4): ReLU(inplace=True)
      )
      (branch2): Sequential(
        (0): Conv2d(24, 58, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(58, eps=1e-05, momentum=0.1, affine=True, track_running_

In [8]:
def load_model():
    # Transfer Learning
    model = models.shufflenet_v2_x1_5(pretrained=pretrained)
    
    # Mode
    model = set_parameter_requires_grad(model, feature_extract)
    
    # Fine tuning
    # Build custom classifier
    model.fc = nn.Linear(in_features=1024,
                        out_features=32)
    return model.to(get_device())

def create_optimizer(model):
    # Parameters to update
    params_to_update = model.parameters()

    if feature_extract:
        params_to_update = []
        for param in model.parameters():
            if param.requires_grad == True:
                params_to_update.append(param)

    else:
        n_params = 0
        for param in model.parameters():
            if param.requires_grad == True:
                n_params += 1


    # Loss function and gradient descent

    criterion = nn.CrossEntropyLoss()

    optimizer = optim.Adam(params_to_update, 
                          lr=0.0001, 
                          weight_decay=0.00004)
    
    print("Running on device:", get_device())
    
    return criterion.to(get_device()), model, optimizer

## Training, validation and test functions

In [9]:
# Variables to store fold scores
train_acc = []
test_top1_acc = []
test_top5_acc = []
test_precision = []
test_recall = []
test_f1 = []
times = []

for fold, (train_idx, valid_idx) in enumerate(splits.split(total_set)):
    
    start_time = time.time()
    
    print('Fold : {}'.format(fold))
    
    # Train and val samplers
    train_sampler = SubsetRandomSampler(train_idx)
    print("Samples in training:", len(train_sampler))
    valid_sampler = SubsetRandomSampler(valid_idx)
    print("Samples in test:", len(valid_sampler))
    
    # Train and val loaders
    train_loader = torch.utils.data.DataLoader(
                      total_set, batch_size=train_batch_size, sampler=train_sampler)
    valid_loader = torch.utils.data.DataLoader(
                      total_set, batch_size=1, sampler=valid_sampler)
    
    device = get_device()
    
    criterion, model, optimizer = create_optimizer(load_model())
    
    # Training
    for epoch in range(h_epochs):
        
        model.train()
        running_loss = 0.0
        running_corrects = 0
        trunning_corrects = 0
        
        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            
            with torch.set_grad_enabled(True):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                
            running_loss += loss.item() * inputs.size(0)
            running_corrects += (preds == labels).sum()
            trunning_corrects += preds.size(0)
            

        epoch_loss = running_loss / trunning_corrects
        epoch_acc = (running_corrects.double()*100) / trunning_corrects
        train_acc.append(epoch_acc.item())
        
        print('\t\t Training: Epoch({}) - Loss: {:.4f}, Acc: {:.4f}'.format(epoch, epoch_loss, epoch_acc))
        
        # Validation
        
        model.eval()  
        
        vrunning_loss = 0.0
        vrunning_corrects = 0
        num_samples = 0
        
        for data, labels in valid_loader:
            
            data = data.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            
            with torch.no_grad():
                outputs = model(data)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
                
            vrunning_loss += loss.item() * data.size(0)
            vrunning_corrects += (preds == labels).sum()
            num_samples += preds.size(0)
            
        vepoch_loss = vrunning_loss/num_samples
        vepoch_acc = (vrunning_corrects.double() * 100)/num_samples
        
        print('\t\t Validation({}) - Loss: {:.4f}, Acc: {:.4f}'.format(epoch, vepoch_loss, vepoch_acc))
    
    # Calculating and appending scores to this fold
    model.class_to_idx = total_set.class_to_idx
    scores = get_scores(model, valid_loader)
    
    test_top1_acc.append(scores[0])
    test_top5_acc.append(scores[1])
    test_precision.append(scores[2])
    test_recall.append(scores[3])
    test_f1.append(scores[4])
    
    time_fold = time.time() - start_time
    times.append(time_fold)
    print("Total time per fold: %s seconds." %(time_fold))

Fold : 0
Samples in training: 21665
Samples in test: 2408
Running on device: cuda
		 Training: Epoch(0) - Loss: 2.0507, Acc: 36.8198
		 Validation(0) - Loss: 1.1446, Acc: 60.2990
		 Training: Epoch(1) - Loss: 1.0095, Acc: 65.5897
		 Validation(1) - Loss: 0.7370, Acc: 75.3738
		 Training: Epoch(2) - Loss: 0.7246, Acc: 75.1489
		 Validation(2) - Loss: 0.6350, Acc: 77.4917
		 Training: Epoch(3) - Loss: 0.5725, Acc: 80.1339
		 Validation(3) - Loss: 0.4991, Acc: 82.6412
		 Training: Epoch(4) - Loss: 0.4730, Acc: 83.3880
		 Validation(4) - Loss: 0.4182, Acc: 85.1329
		 Training: Epoch(5) - Loss: 0.3963, Acc: 86.2128
		 Validation(5) - Loss: 0.4228, Acc: 85.5066
		 Training: Epoch(6) - Loss: 0.3540, Acc: 87.6298
		 Validation(6) - Loss: 0.3839, Acc: 86.4618
		 Training: Epoch(7) - Loss: 0.3141, Acc: 89.0699
		 Validation(7) - Loss: 0.4792, Acc: 83.9701
		 Training: Epoch(8) - Loss: 0.2789, Acc: 90.3162
		 Validation(8) - Loss: 0.4050, Acc: 86.9186
		 Training: Epoch(9) - Loss: 0.2547, Acc: 91

		 Training: Epoch(1) - Loss: 0.9555, Acc: 67.2251
		 Validation(1) - Loss: 0.7040, Acc: 75.0312
		 Training: Epoch(2) - Loss: 0.6962, Acc: 75.6993
		 Validation(2) - Loss: 0.5711, Acc: 80.5152
		 Training: Epoch(3) - Loss: 0.5563, Acc: 80.9056
		 Validation(3) - Loss: 0.4154, Acc: 85.1683
		 Training: Epoch(4) - Loss: 0.4596, Acc: 84.0857
		 Validation(4) - Loss: 0.3722, Acc: 86.7470
		 Training: Epoch(5) - Loss: 0.3941, Acc: 86.4811
		 Validation(5) - Loss: 0.4332, Acc: 84.3789
		 Training: Epoch(6) - Loss: 0.3378, Acc: 88.2212
		 Validation(6) - Loss: 0.2854, Acc: 90.1537
		 Training: Epoch(7) - Loss: 0.2906, Acc: 90.1135
		 Validation(7) - Loss: 0.3152, Acc: 88.8658
		 Training: Epoch(8) - Loss: 0.2636, Acc: 90.8197
		 Validation(8) - Loss: 0.3212, Acc: 88.4504
		 Training: Epoch(9) - Loss: 0.2395, Acc: 91.7197
		 Validation(9) - Loss: 0.3438, Acc: 88.1595
		 Training: Epoch(10) - Loss: 0.2114, Acc: 92.9613
		 Validation(10) - Loss: 0.2945, Acc: 89.1982
		 Training: Epoch(11) - Los

In [10]:
print("Train accuracy average: ", np.mean(train_acc) / 100)
print("Top-1 test accuracy average: ", np.mean(test_top1_acc))
print("Top-5 test accuracy average: ", np.mean(test_top5_acc))
print("Weighted Precision test accuracy average: ", np.mean(test_precision))
print("Weighted Recall test accuracy average: ", np.mean(test_recall))
print("Weighted F1 test accuracy average: ", np.mean(test_f1))
print("Average time per fold (seconds):", np.mean(times))

Train accuracy average:  0.8426502062052831
Top-1 test accuracy average:  0.9001793460932745
Top-5 test accuracy average:  0.9968014456727126
Weighted Precision test accuracy average:  0.9073827126091827
Weighted Recall test accuracy average:  0.9001793460932745
Weighted F1 test accuracy average:  0.9000450217544824
Average time per fold (seconds): 1283.9797766447068
